## facematch

Aqui, espero conseguir mesmo que de maneira rudimentar encontrar os vetores que represetam os dois rostos. Para isso, pretendo utilizar o 'pacote' https://github.com/DefTruth/torchlm.

In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 1.7 ms (started: 2024-01-03 15:56:02 -03:00)


In [3]:
#load parquet

df = pd.read_parquet('../base/filtered_data.parquet')

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5173 entries, 0 to 23593
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  5173 non-null   object
 1   approved            5173 non-null   bool  
 2   analysis_status     5154 non-null   object
 3   kind                5173 non-null   object
 4   month               5173 non-null   int32 
 5   registration_image  5173 non-null   object
 6   comparision_image   5173 non-null   object
dtypes: bool(1), int32(1), object(5)
memory usage: 267.7+ KB
time: 100 ms (started: 2024-01-03 15:29:46 -03:00)


In [20]:
import face_alignment
from skimage import io

fa = face_alignment.FaceAlignment(face_alignment.LandmarksType.TWO_D, flip_input=False, device = 'cpu', face_detector='blazeface')

input = io.imread('imagens_test/biden.png')
landmarks, landmarks_scores, detected_faces = fa.get_landmarks(input, return_landmark_score=True)


,id,approved,analysis_status,kind,month,registration_image,comparision_image
index,,,,,,,
0,c1f483cd-0ead-4c83-bd7f-2434e89cd7ba,True,BIOMETRIC_AUTHENTICATED,MANUAL_BY_BIOMETRIC_FACE,10,c1f483cd-0ead-4c83-bd7f-2434e89cd7ba_selfie.jpeg,c1f483cd-0ead-4c83-bd7f-2434e89cd7ba_transacti...
1,81e61bb8-dcde-4334-a43c-a9d4d7b03a4b,True,BIOMETRIC_TRANSACTION_BAD_QUALITY,MANUAL_BY_BIOMETRIC_FACE,10,81e61bb8-dcde-4334-a43c-a9d4d7b03a4b_selfie.jpeg,81e61bb8-dcde-4334-a43c-a9d4d7b03a4b_transacti...
2,9be3c172-d260-4def-a941-eb0e2761becd,True,BIOMETRIC_AUTHENTICATED,MANUAL_BY_BIOMETRIC_FACE,9,9be3c172-d260-4def-a941-eb0e2761becd_selfie.jpeg,9be3c172-d260-4def-a941-eb0e2761becd_transacti...
3,83e57219-6fae-4bb7-8e67-2b0271fc1adb,True,BIOMETRIC_AUTHENTICATED,MANUAL_BY_BIOMETRIC_FACE,10,83e57219-6fae-4bb7-8e67-2b0271fc1adb_selfie.jpeg,83e57219-6fae-4bb7-8e67-2b0271fc1adb_transacti...
4,e23886d9-755f-4b1c-a523-50561f6493a7,True,BIOMETRIC_NEEDS_REVALIDATION,MANUAL_BY_BIOMETRIC_FACE,10,e23886d9-755f-4b1c-a523-50561f6493a7_selfie.jpeg,e23886d9-755f-4b1c-a523-50561f6493a7_transacti...


time: 398 ms (started: 2024-01-03 15:47:59 -03:00)


In [24]:
import face_alignment
from skimage import io
import pandas as pd
import numpy as np

# Initialize face_alignment
fa = face_alignment.FaceAlignment(face_alignment.LandmarksType.TWO_D, flip_input=False, device='cpu', face_detector='blazeface')

# Function to get landmarks and their scores
def get_landmarks(image_path):
    image = io.imread(image_path)
    landmarks, scores, _ = fa.get_landmarks(image, return_landmark_score=True)
    if landmarks is not None:
        avg_score = np.mean(scores)
        return landmarks[0], avg_score  # Assuming one face per image
    else:
        return None, None

# Function to align images based on eye landmarks and compare
def align_and_compare(landmarks1, landmarks2):
    # Define alignment and comparison logic here
    # For simplicity, this function will just calculate the Euclidean distance
    # between corresponding landmarks as a basic comparison metric
    if landmarks1 is not None and landmarks2 is not None:
        distances = np.linalg.norm(landmarks1 - landmarks2, axis=1)
        avg_distance = np.mean(distances)
        return avg_distance
    else:
        return None


# New columns for scores and comparison
df['registration_avg_score'] = np.nan
df['comparison_avg_score'] = np.nan
df['facematch_score'] = np.nan

# Iterate over DataFrame
for index, row in tqdm.tqdm(df.iterrows()):
    reg_landmarks, reg_score = get_landmarks('../base/imagens/'+row['registration_image'])
    comp_landmarks, comp_score = get_landmarks('../base/imagens/'+row['comparision_image'])

    df.at[index, 'registration_avg_score'] = reg_score
    df.at[index, 'comparison_avg_score'] = comp_score

    if reg_landmarks is not None and comp_landmarks is not None:
        facematch_score = align_and_compare(reg_landmarks, comp_landmarks)
        df.at[index, 'facematch_score'] = facematch_score

# Display the updated DataFrame
print(df)


136it [08:10,  5.15s/it]

In [ ]:
df.to_parquet('../base/scored_base.parquet')
